<a href="https://colab.research.google.com/github/PranavShashidhara/Netflix-Recommendation-system/blob/main/Dictionaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.utils import shuffle
import boto3
from collections import Counter

In [3]:
data_df = pd.read_parquet('https://netflix-project-data.s3.us-east-1.amazonaws.com/Preprocessed+data/small_filtered_data.parquet')
data_df.head()

,User_id,Movie_id,Ratings,Date,RemappedUserId
0,2456613,8,3,2004-03-31,292668
1,1979083,8,2,2004-05-12,196704
2,1391523,8,3,2004-04-27,78623
3,1744889,8,1,2005-08-25,149742
4,995497,8,2,2005-02-14,479264


In [48]:
# Remapping both user Id's and Movie'Ids
user_ids_count = Counter(data_df.RemappedUserId)
movie_ids_count = Counter(data_df.RemappedUserId)
n = 10000
m = 2000
user_ids = data_df['RemappedUserId'].unique()
movie_ids = data_df['Movie_id'].unique()

data_df['RemappedUserId'] = data_df['RemappedUserId'].astype(int)
data_df['Movie_id'] = data_df['Movie_id'].astype(int)
df_small = data_df.copy()

new_user_id_map = {}
i = 0
for old in user_ids:
    new_user_id_map[old] = i
    i += 1
print("i:", i)

new_movie_id_map = {}
j = 0
for old in movie_ids:
  new_movie_id_map[old] = j
  j += 1
print("j:", j)

print("Setting new ids")
df_small.loc[:, 'User_id'] = df_small.apply(lambda row: new_user_id_map[row.RemappedUserId], axis=1)
df_small.loc[:, 'Movie_id'] = df_small.apply(lambda row: new_movie_id_map[row.Movie_id], axis=1)

print("Max user id:", df_small.User_id.max())
print("Max movie id:", df_small.Movie_id.max())

print("Small dataframe size:",len(df_small))

i: 10000
j: 2000
Setting new ids
Max user id: 9999
Max movie id: 1999
Small dataframe size: 9357999


In [39]:
user_ids[0]

339651

In [49]:
df_small.min()

,0
User_id,0
Movie_id,0
Ratings,1
Date,1999-12-21
RemappedUserId,37


In [27]:
data_df.dtypes

,0
User_id,int64
Movie_id,int64
Ratings,object
Date,object
RemappedUserId,int32


In [28]:
data_df.columns

Index(['User_id', 'Movie_id', 'Ratings', 'Date', 'RemappedUserId'], dtype='object')

In [29]:
df_small

,User_id,Movie_id,Ratings,Date,RemappedUserId
528569,4064,427,4,2005-05-10,350840
529120,7698,427,2,2004-08-28,342763
529211,683,427,4,2005-07-16,95045
529526,9002,427,3,2003-12-30,345223
529591,7510,427,2,2005-04-25,469760
...,...,...,...,...,...
8036229,6647,1472,5,2004-06-21,21039
8036900,8428,1472,5,2003-10-22,154601
8037656,4641,1472,3,2005-03-03,279593
8037881,3893,1472,2,2003-11-13,286703


In [50]:
df_small['Movie_id'] = pd.to_numeric(df_small['Movie_id'], errors='coerce')

print(f"Minimum user ID: {df_small['User_id'].min()}")
print(f"Minimum movie ID: {df_small['Movie_id'].min()}")


print(f"Max user ID: {df_small['User_id'].max()}")
print(f"Max movie ID: {df_small['Movie_id'].max()}")

Minimum user ID: 0
Minimum movie ID: 0
Max user ID: 9999
Max movie ID: 1999


In [51]:
N = df_small.RemappedUserId.max()
df_small.Movie_id.max()
df_small = shuffle(df_small)
cutoff = int(0.8*len(df_small))
df_train = df_small.iloc[:cutoff]
df_test = df_small.iloc[cutoff:]

user2movie = {}
movie2user = {}
usermovie2rating = {}
count = 0
def update_user2movie_and_movie2user(row):
  global count
  count +=1
  if count % 100000 == 0:
    print("preprocessed %.3f" % (float(count)/cutoff))
  #print(row)
  i = int(row.RemappedUserId)
  j = int(row.Movie_id)
  if i not in user2movie:
    user2movie[i] = [j]
  else:
    user2movie[i].append(j)

  if j not in movie2user:
    movie2user[j] = [i]
  else:
    movie2user[j].append(i)
  usermovie2rating[(i, j)] = row.Ratings
df_train.apply(update_user2movie_and_movie2user, axis=1)

print("Calling: update_usermovie2rating_test")
count = 0
usermovie2rating_test = {}
def update_usermovie2rating(row):
  global count
  count +=1
  i = int(row.RemappedUserId)
  j = int(row.Movie_id)
  usermovie2rating_test[(i, j)] = row.Ratings

df_test.apply(update_usermovie2rating, axis=1)

preprocessed 0.013
preprocessed 0.027
preprocessed 0.040
preprocessed 0.053
preprocessed 0.067
preprocessed 0.080
preprocessed 0.094
preprocessed 0.107
preprocessed 0.120
preprocessed 0.134
preprocessed 0.147
preprocessed 0.160
preprocessed 0.174
preprocessed 0.187
preprocessed 0.200
preprocessed 0.214
preprocessed 0.227
preprocessed 0.240
preprocessed 0.254
preprocessed 0.267
preprocessed 0.281
preprocessed 0.294
preprocessed 0.307
preprocessed 0.321
preprocessed 0.334
preprocessed 0.347
preprocessed 0.361
preprocessed 0.374
preprocessed 0.387
preprocessed 0.401
preprocessed 0.414
preprocessed 0.427
preprocessed 0.441
preprocessed 0.454
preprocessed 0.468
preprocessed 0.481
preprocessed 0.494
preprocessed 0.508
preprocessed 0.521
preprocessed 0.534
preprocessed 0.548
preprocessed 0.561
preprocessed 0.574
preprocessed 0.588
preprocessed 0.601
preprocessed 0.614
preprocessed 0.628
preprocessed 0.641
preprocessed 0.655
preprocessed 0.668
preprocessed 0.681
preprocessed 0.695
preprocessed

,0
32088,None
470478,None
4749095,None
3955496,None
3974615,None
...,...
553429,None
6199731,None
112435,None
8086790,None


In [52]:
with open('user2movie.json', 'wb') as f:
  pickle.dump(user2movie, f)

with open('movie2user.json', 'wb') as f:
  pickle.dump(movie2user, f)

with open('usermovie2rating.json', 'wb') as f:
  pickle.dump(usermovie2rating, f)

with open('usermovie2rating_test.json', 'wb') as f:
  pickle.dump(usermovie2rating_test, f)